In [5]:
# imports
import time

In [6]:
# Util functions
def time_to_minutes(time):
    time = time.split(':')
    minutes = int(time[0]) * 60 + int(time[1])
    return minutes


def minutes_to_time(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return '{}:{:02d}'.format(hours, minutes)


def path_to_str(path):
    return path.__str__().replace("['", '').replace("']", '').replace("', '", " -> ")


def time_function(func, *args):
    start = time.perf_counter()
    func(*args)
    end = time.perf_counter()
    print(f'Tiempo de ejecución: {end - start:.8f} segundos')

In [7]:
# Class Graph
class Graph:
    def __init__(self):
        self.adjency_table = []
        # read tables from csv files
        with open('distance.csv', 'r') as distance_file, open('time.csv', 'r') as time_file:
            distance_row = distance_file.readline()
            time_row = time_file.readline()

            self.cities = distance_row.strip().replace('"', '').split(',')[1:]

            for city in self.cities:
                distance_row = distance_file.readline()
                time_row = time_file.readline()

                if not distance_row or not time_row:
                    break

                distance_row = distance_row.strip().replace('"', '').split(',')[1:]
                time_row = time_row.strip().replace('"', '').split(',')[1:]

                self.adjency_table.append([(int(x) if x != '' else 0, time_to_minutes(y) if y != '' else 0) for x, y in zip(distance_row, time_row)])

    def shortest_path_dynamic(self, city1, city2):
        dinamic_table = []

        # Create copy of cities withouth city1 and city2
        sorted_cities = self.cities.copy()
        sorted_cities.pop(sorted_cities.index(city1))
        sorted_cities.pop(sorted_cities.index(city2))

        # Set the first row to city1 row
        dinamic_table.append(self.adjency_table[self.cities.index(city1)])

        for i, city in enumerate(sorted_cities, start=1):
            index = self.cities.index(city)
            dinamic_table.append([0 for _ in range(len(dinamic_table[0]))])
            for j, (distance, time) in enumerate(self.adjency_table[index]):
                new_distance = min(dinamic_table[i - 1][j][0], dinamic_table[i - 1][index][0] + distance)
                new_time = min(dinamic_table[i - 1][j][1], dinamic_table[i - 1][index][1] + time)
                dinamic_table[i][j] = (new_distance, new_time)

        # get path from dinamic table
        distance_path = []
        time_path = []
        distance_path.append(city2)
        time_path.append(city2)
        for i, row in reversed(list(enumerate(dinamic_table))):
            index = self.cities.index(distance_path[-1])
            if row[index][0] != dinamic_table[i - 1][index][0]:
                distance_path.append(sorted_cities[i - 1])
            if row[index][1] != dinamic_table[i - 1][index][1]:
                time_path.append(sorted_cities[i - 1])
        distance_path.append(city1)
        distance_path.reverse()
        time_path.append(city1)
        time_path.reverse()

        shortest_distance, shortest_time = dinamic_table[-1][self.cities.index(city2)]
        return shortest_distance, minutes_to_time(shortest_time), distance_path, time_path

In [8]:
def main():
    graph = Graph()
    city1 = 'Armenia'
    city2 = 'Medellín'
    d, t, dp, tp = graph.shortest_path_dynamic(city1, city2)
    print(f'{path_to_str(dp)} = {d} km')
    print(f'{path_to_str(tp)} = {t}')

    # time function
    time_function(graph.shortest_path_dynamic, city1, city2)

if __name__ == '__main__':
    main()

Armenia -> Pereira -> Medellín = 279 km
Armenia -> Medellín = 3:49
Tiempo de ejecución: 0.00011321 segundos
